# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key


In [32]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
df = pd.read_csv("../Output_Data/cities.csv", index_col=False)
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,samarai,-10.6167,150.6667,83.61,75.0,100.0,11.16,PG,1.614646e+09
1,2,zhangye,38.9342,100.4517,30.34,46.0,15.0,2.51,CN,1.614646e+09
2,3,tilichiki,60.4667,166.1000,-7.02,83.0,95.0,8.08,RU,1.614646e+09
3,4,carnaiba,-8.7000,-37.6667,73.80,80.0,22.0,11.68,BR,1.614646e+09
4,5,dikson,73.5069,80.5464,-20.15,90.0,2.0,14.00,RU,1.614646e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Google developer API key
# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [18]:
#store lat and lng
locations = df[["Lat","Lng"]]
humidity = df["Humidity"]

#heatmap
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': 'auto'
}
heatmap = gmaps.figure(center = [0,0] ,zoom_level = 2, layout=figure_layout)

#add layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
heatmap.add_layer(heat_layer)

heatmap


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='auto', padding='1px', width='1000…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
temp_df = df.loc[(df["Max Temp"] < 80) & (df["Max Temp"] >70) ]
weather_criteria = temp_df.loc[(temp_df["Wind Speed"] < 10) & (temp_df["Cloudiness"] == 0)]
weather_criteria


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,22,grand gaube,-20.0064,57.6608,75.99,78.0,0.0,1.99,MU,1.614646e+09
26,30,felipe carrillo puerto,19.5833,-88.0500,75.06,79.0,0.0,4.47,MX,1.614646e+09
54,61,yeola,20.0333,74.4833,72.75,20.0,0.0,9.46,IN,1.614646e+09
59,66,ouadda,8.0777,22.4007,73.69,19.0,0.0,7.81,CF,1.614646e+09
74,81,saint-joseph,-21.3667,55.6167,75.20,57.0,0.0,6.91,RE,1.614646e+09
78,86,dagana,16.5165,-15.5068,75.20,29.0,0.0,2.30,MR,1.614646e+09
101,110,saint-philippe,-21.3585,55.7679,75.20,57.0,0.0,6.91,RE,1.614646e+09
124,135,ler,8.3018,30.1418,76.78,18.0,0.0,6.15,SS,1.614646e+09
156,169,bentiu,9.2333,29.8333,75.24,18.0,0.0,7.67,SS,1.614646e+09
157,170,salalah,17.0151,54.0924,71.60,43.0,0.0,5.75,OM,1.614646e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
